In [32]:
#!/usr/bin/env python

# Import Required Packages
# ========================
import os, sys
import math
import pickle

import casadi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

from gams import *
import os
import sys

sys.path.append(os.path.abspath("mcmc"))
from mcmc_sampling import create_hmc_sampler

# Local Debugging flag; remove when all tested
_DEBUG = True 


In [33]:
from gams import GamsWorkspace
if GamsWorkspace.api_major_rel_number<42:  # old API structure
    import gdxcc as gdx
    from gams import *
    import gamstransfer as gt
else:  # new API structure
    import gams.core.gdx as gdx
    from gams.control import *
    import gams.transfer as gt

In [34]:
def multivariate_normal_log_probability(z, mean, cov):
    """
    Evaluate and return the log-probability of a (unnormalized) multivariate normal distribution.

    :param z: value of the multivariate normal random variable
    :param mean: (1d array/list); mean of the normal distribution
    :param cov: (2D array) covariance matrix

    :returns: log-probability of a (unnormalized) multivariate normal distribution
    """
    z    = np.asarray(z, dtype=float).flatten()
    mean = np.asarray(z, dtype=float).flatten()
    cov = np.asarray(cov, dtype=float).squeeze()

    # Assert/Check input types/shapes
    assert mean.size == z.size, "mismatch between variable and mean sizes/shapes"
    if mean.size > 1:
        assert cov.shape == (z.size,z.size), "mismatch between variable and covariances `cov` shape"

    elif mean.size == 1:
        assert cov.size == 1, "mismatch between variable and covariances `cov` shape"
        cov = np.reshape(cov, (1, 1))

    else:
        print("Invalid covariance matrix shape!")
        raise AssertionError

    # Evaluate the log-probability
    dev      = z - mean
    scld_dev = np.linalg.inv(cov).dot(dev)
    log_prob = - 0.5 * np.dot(dev, scld_dev)

    return log_prob

In [35]:
site_num = 25

In [36]:
def load_site_data(site_num, 
                   norm_fac=1e6,
                  ):
    """
    Load site data

    :returns:
        -
    """
    # Read data file
    n=site_num
    file=f"data/calibration_{n}SitesModel.csv"
    df = pd.read_csv(file)

    
    # Extract information
    z_2017             = df[f'z_2017_{n}Sites'].to_numpy()
    zbar_2017          = df[f'zbar_2017_{n}Sites'].to_numpy()
    gamma              = df[f'gamma_{n}Sites'].to_numpy()
    gammaSD            = df[f'gammaSD_{n}Sites'].to_numpy()
    forestArea_2017_ha = df[f'forestArea_2017_ha_{n}Sites'].to_numpy()
    theta              = df[f'theta_{n}Sites'].to_numpy()

    # Normalize Z data
    zbar_2017 /= norm_fac
    z_2017    /= norm_fac
    
    return (zbar_2017,
            gamma,
            gammaSD,
            z_2017,
            forestArea_2017_ha,
            theta
            )

In [37]:
# Verbose Data check (eyeball check!)
zbar_2017, gamma, gammaSD, \
            z_2017, forestArea_2017_ha, theta = load_site_data(site_num,norm_fac=1e+9)
print("gamma", gamma)
print("gammaSD", gammaSD)
print("z_2017", z_2017)
print("gamma - gammaSD", gamma - gammaSD)
print("zbar_2017", zbar_2017)
print("theta", theta)

gammadata = pd.DataFrame(gamma)

gammadata.to_csv('GammaData.csv', index=False)


gamma [516.88557127 488.95668786 535.7933372  614.38476229 697.34085963
 594.38440765 540.18969668 502.09011583 586.13789618 622.92870153
 468.53543538 216.29945459 736.1126944  644.37719596 568.13204882
 618.56342492 570.93769461 376.7589174  806.29456683 649.96115287
 577.10631912 455.99483831 366.09394675 337.21852297 287.82630347]
gammaSD [22.01380318 12.30650843 23.34479384 32.19887999 33.0552216  20.86507363
 10.44879734  7.97210675 16.35650086 18.89356099  5.66673842  4.7426229
 27.52295223 16.35653107 10.55158851 21.85866557 24.68789648  7.04434639
 35.21752443 25.01272143 13.69821009  6.68635044  1.78628409 14.31382086
 17.20154406]
z_2017 [5.77092940e-07 5.58918518e-06 7.87674083e-04 9.64686174e-06
 1.28634503e-04 1.62486537e-05 7.71270596e-05 4.13615376e-04
 1.41563731e-03 2.97549040e-03 7.91141025e-03 3.11951500e-04
 4.91859804e-04 5.48889538e-04 1.19367249e-03 1.41661798e-03
 6.05514368e-03 6.74644202e-03 8.44104902e-05 2.44820053e-03
 7.38590342e-03 8.45649924e-03 5.22554

In [38]:
def log_density_function(gamma_val,
                         gamma_vals_mean,
                         theta_vals,
                         site_precisions,
                         alpha,
                         X,
                         Ua,
                         Up,
                         zbar_2017,
                         forestArea_2017_ha,
                         norm_fac,
                         alpha_p_Adym,
                         Bdym,
                         leng,
                         T,
                         ds_vect,
                         zeta,
                         xi,
                         kappa,
                         pa,
                         pf,
                         ):
    """
    Define a function to evaluate log-density of the objective/posterior distribution
    Some of the input parameters are updated at each cycle of the outer loop (optimization loop),
    and it becomes then easier/cheaper to udpate the function stamp and keep it separate here
    """
    N          = X.shape[1] - 1
    
    gamma_val  = np.asarray(gamma_val).flatten()
    gamma_size = gamma_val.size
    x0_vals    = gamma_val.T.dot(forestArea_2017_ha) / norm_fac
    X_zero     = np.sum(x0_vals) * np.ones(leng)
    
    
    # shifted_X = zbar_2017 -  (X)[0:gamma_size, :-1]
    shifted_X  = (X)[0: gamma_size, :-1].copy()
    for j in range(N): 
        shifted_X[:, j]  = zbar_2017 - shifted_X[:, j]
    omega      = np.dot(gamma_val, alpha * shifted_X -  (Up)[0:-1].T)
    
    X_dym      = np.zeros(T+1)
    X_dym[0]   = np.sum(x0_vals)
    X_dym[1: ] = alpha_p_Adym * X_zero  + np.dot(Bdym, omega.T)

    z_shifted_X =  (X)[0: gamma_size, :].copy()
    scl = pa * theta_vals - pf * kappa
    for j in range(N+1): z_shifted_X [:, j] *= scl
    term_1 = - casadi.sum2(np.reshape(ds_vect[0: T], (1, T)) * (Ua)[0:-1].T * zeta / 2 )
    term_2 = casadi.sum2(np.reshape(ds_vect[0: T], (1, T)) * pf * (X_dym[1: ] - X_dym[0: -1]))
    term_3 = casadi.sum2(np.reshape(ds_vect, (1, N+1)) * casadi.sum1(z_shifted_X))
    obj_val = (term_1 + term_2 + term_3)

    gamma_val_dev   = gamma_val - gamma_vals_mean
    norm_log_prob   =   - 0.5 * np.dot(gamma_val_dev,
                                       site_precisions.dot(gamma_val_dev)
                                       )
    log_density_val = -1.0  / xi * obj_val + norm_log_prob
    if _DEBUG:
        print("Term 1: ", term_1)
        print("Term 2: ", term_2)
        print("Term 3: ", term_3)
        print("obj_val: ", obj_val)
        print("norm_log_prob", norm_log_prob)
        print("log_density_val", log_density_val)
        print("gamma_value", gamma_val)

    return log_density_val

In [41]:
def main(
    # Configurations/Settings
    norm_fac          = 1e6,
    delta_t           = 0.02,
    alpha             = 0.045007414,
    kappa             = 2.094215255,
    pf                = 20.76,
    pa                = 44.75,
    xi                = 10,
    zeta              = 1.66e-4*1e6,# zeta := 1.66e-4*norm_fac  #
    #
    max_iter          = 200,
    tol               = 0.01,
    T                 = 200,
    N                 = 200,
    #
    sample_size       = 1000,    # simulations before convergence (to evaluate the mean)
    mode_as_solution  = False,   # If true, use the mode (point of high probability) as solution for gamma
    final_sample_size = 100_00, # number of samples to collect after convergence
    ):
    """
    Main function; putting things together

    :param float tol: convergence tolerance
    :param T:
    :param N:
    """
    

    # Load sites' data
    zbar_2017, gamma, gammaSD, \
        z_2017, forestArea_2017_ha, theta \
        = load_site_data(site_num,norm_fac=norm_fac)


    # Evaluate Gamma values ()
    gamma_1_vals  = gamma -  gammaSD
    gamma_2_vals  = gamma +  gammaSD
    gamma_size    = gamma.size

    # Evaluate mean and covariances from site data
    site_stdev       = gammaSD
    site_covariances = np.diag(np.power(site_stdev, 2))
    site_precisions  = np.linalg.inv(site_covariances)
    site_mean        = gamma_1_vals/2 + gamma_2_vals/2

    # Retrieve z data for selected site(s)
    site_z_vals  = z_2017

    # Initialize Gamma Values
    gamma_vals      = gamma.copy()
    gamma_vals_mean = gamma.copy()
    gamma_vals_old  = gamma.copy()

    # Theta Values
    theta_vals  = theta

    # Householder to track sampled gamma values
    # gamma_vals_tracker       = np.empty((gamma_vals.size, sample_size+1))
    # gamma_vals_tracker[:, 0] = gamma_vals.copy()
    gamma_vals_tracker = [gamma_vals.copy()]

    # Collected Ensembles over all iterations; dictionary indexed by iteration number
    collected_ensembles = {}

    # Track error over iterations
    error_tracker = []

    # Update this parameter (leng) once figured out where it is coming from
    leng = 200
    arr  = np.cumsum(
             np.triu(
             np.ones((leng, leng))
         ),
         axis=1,
    ).T
    Bdym         = (1-alpha) ** (arr-1)
    Bdym[Bdym>1] = 0.0
    Adym         = np.arange(1, leng+1)
    alpha_p_Adym = np.power(1-alpha, Adym)

    # Initialize Blocks of the A matrix those won't change
    A  = np.zeros((gamma_size+2, gamma_size+2))
    Ax = np.zeros(gamma_size+2)

    # Construct Matrix B
    B = np.eye(N=gamma_size+2, M=gamma_size, k=0)
    B = casadi.sparsify(B)

    # Construct Matrxi D constant blocks
    D  = np.zeros((gamma_size+2, gamma_size))

    # time step!
    dt = T / N

    # Other placeholders!
    ds_vect = np.exp(- delta_t * np.arange(N+1) * dt)
    ds_vect = np.reshape(ds_vect, (ds_vect.size, 1))

    # Results dictionary
    results = dict(
        gamma_size=gamma_size,
        tol=tol,
        T=T,
        N=N,
        norm_fac=norm_fac,
        delta_t=delta_t,
        alpha=alpha,
        kappa=kappa,
        pf=pf,
        pa=pa,
        xi=xi,
        zeta=zeta,
        sample_size=sample_size,
        final_sample_size=final_sample_size,
        mode_as_solution=mode_as_solution,
    )

    # Initialize error & iteration counter
    error = np.infty
    cntr = 0

    # Loop until convergence
    while cntr < max_iter and error > tol:

        # Update x0
        x0_vals = gamma_vals * forestArea_2017_ha
        
        x0data = pd.DataFrame(x0_vals)
        x0data.to_csv('X0Data.csv')
        
        gammadata = pd.DataFrame(gamma_vals)

        gammadata.to_csv('GammaData.csv')
        
        cwd = os.getcwd() # get current working directory

        ws = GamsWorkspace(system_directory=r"C:\GAMS\43", working_directory=cwd)
        t1 = ws.add_job_from_file("amazon_25sites.gms")
        t1.run()
        dfu = pd.read_csv('amazon_data_u.dat', delimiter='\t')
        # Process the data using the pandas DataFrame
        dfu=dfu.drop('T/R ', axis=1)
        dfu_np =dfu.to_numpy()

        dfv = pd.read_csv('amazon_data_v.dat', delimiter='\t')
        # Process the data using the pandas DataFrame
        dfv=dfv.drop('T/R ', axis=1)
        dfv_np =dfv.to_numpy()

        dfw = pd.read_csv('amazon_data_w.dat', delimiter='\t')
        # Process the data using the pandas DataFrame
        dfw =dfw.drop('T   ', axis=1)
        dfw_np = dfw.to_numpy()

        dfx = pd.read_csv('amazon_data_x.dat', delimiter='\t')
        # Process the data using the pandas DataFrame
        dfx =dfx.drop('T   ', axis=1)
        dfx_np = dfx.to_numpy()

        dfz = pd.read_csv('amazon_data_z.dat', delimiter='\t')
        # Process the data using the pandas DataFrame
        dfz=dfz.drop('T/R ', axis=1)
        dfz_np =dfz.to_numpy()

        Ua = dfw_np**2
        X_value = np.concatenate((dfz_np.T, dfx_np.T))
        """         if _DEBUG:
            print("site_z_vals: ", site_z_vals)
            print("x0_vals: ", x0_vals)
            print("casadi.vertcat(site_z_vals,np.sum(x0_vals),1): ", casadi.vertcat(site_z_vals,np.sum(x0_vals),1))

        if _DEBUG:
            print(" (X)",  (X_value))
            print(" (Ua)",  (Ua))
            print(" (Up)",  (dfu_np))
            print(" (Um)",  (dfv_np)) """
        
        ## Start Sampling
        # Update signature of log density evaluator
        print(log_density_function(gamma_val=gamma_vals,
                                                             gamma_vals_mean=gamma_vals_mean,
                                                             theta_vals=theta_vals,
                                                             site_precisions=site_precisions,
                                                             alpha=alpha,
                                                             X=X_value,
                                                             Ua=Ua,
                                                             Up=dfu_np,
                                                             zbar_2017=zbar_2017,
                                                             forestArea_2017_ha=forestArea_2017_ha,
                                                             norm_fac=norm_fac,
                                                             alpha_p_Adym=alpha_p_Adym,
                                                             Bdym=Bdym,
                                                             leng=leng,
                                                             T=T,
                                                             ds_vect=ds_vect,
                                                             zeta=zeta,
                                                             xi=xi,
                                                             kappa=kappa,
                                                             pa=pa,
                                                             pf=pf,
                                                             ))
        log_density = lambda gamma_val: log_density_function(gamma_val=gamma_val,
                                                             gamma_vals_mean=gamma_vals_mean,
                                                             theta_vals=theta_vals,
                                                             site_precisions=site_precisions,
                                                             alpha=alpha,
                                                             X=X_value,
                                                             Ua=Ua,
                                                             Up=dfu_np,
                                                             zbar_2017=zbar_2017,
                                                             forestArea_2017_ha=forestArea_2017_ha,
                                                             norm_fac=norm_fac,
                                                             alpha_p_Adym=alpha_p_Adym,
                                                             Bdym=Bdym,
                                                             leng=leng,
                                                             T=T,
                                                             ds_vect=ds_vect,
                                                             zeta=zeta,
                                                             xi=xi,
                                                             kappa=kappa,
                                                             pa=pa,
                                                             pf=pf,
                                                             )

        # Create MCMC sampler & sample, then calculate diagnostics
        sampler = create_hmc_sampler(
            size=gamma_size,
            log_density=log_density,
            #
            burn_in=100,
            mix_in=2,
            symplectic_integrator='verlet',
            symplectic_integrator_stepsize=1e-1,
            symplectic_integrator_num_steps=3,
            mass_matrix=1,
            constraint_test=lambda x: True if np.all(x>=0) else False,
        )
        gamma_post_samples = sampler.sample(
            sample_size=sample_size,
            initial_state=gamma_vals,
            verbose=True,
        )
        gamma_post_samples = np.asarray(gamma_post_samples)

        # Update ensemble/tracker
        collected_ensembles.update({cntr: gamma_post_samples.copy()})

        # Update gamma value
        weight     = 0.25  # <-- Not sure how this linear combination weighting helps!
        if mode_as_solution:
            raise NotImplementedError("We will consider this in the future; trace sampled points and keep track of objective values to pick one with highest prob. ")
            
        else:
            gamma_vals = weight * np.mean(gamma_post_samples, axis=0 ) + (1-weight) * gamma_vals_old
        gamma_vals_tracker.append(gamma_vals.copy())

        # Evaluate error for convergence check
        error = np.max(np.abs(gamma_vals_old-gamma_vals) / gamma_vals_old)
        error_tracker.append(error)
        print(f"Iteration [{cntr+1:4d}]: Error = {error}")

        # Exchange gamma values (for future weighting/update & error evaluation)
        gamma_vals_old = gamma_vals

        # Increase the counter
        cntr += 1

        results.update({'cntr': cntr,
                        'error_tracker':np.asarray(error_tracker),
                        'gamma_vals_tracker': np.asarray(gamma_vals_tracker),
                        'collected_ensembles':collected_ensembles,
                        })
        pickle.dump(results, open('results.pcl', 'wb'))
        
        # Extensive plotting for monitoring; not needed really!
        if False:
            plt.plot(gamma_vals_tracker[-2], label=r'Old $\gamma$')
            plt.plot(gamma_vals_tracker[-1], label=r'New $\gamma$')
            plt.legend()
            plt.show()

            for j in range(gamma_size):
                plt.hist(gamma_post_samples[:, j], bins=50)
                plt.title(f"Iteration {cntr}; Site {j+1}")
                plt.show()
    
    print("Terminated. Sampling the final distribution")
    # Sample (densly) the final distribution
    final_sample = sampler.sample(
        sample_size=final_sample_size,
        initial_state=gamma_vals,
        verbose=True,
    )
    final_sample = np.asarray(final_sample)
    results.update({'final_sample': final_sample})
    pickle.dump(results, open('results.pcl', 'wb'))
    
    return results


In [42]:
results = main()

Term 1:  -13888
Term 2:  336923
Term 3:  36342.7
obj_val:  359378
norm_log_prob -0.0
log_density_val -35937.8
gamma_value [516.88557127 488.95668786 535.7933372  614.38476229 697.34085963
 594.38440765 540.18969668 502.09011583 586.13789618 622.92870153
 468.53543538 216.29945459 736.1126944  644.37719596 568.13204882
 618.56342492 570.93769461 376.7589174  806.29456683 649.96115287
 577.10631912 455.99483831 366.09394675 337.21852297 287.82630347]
-35937.8
Term 1:  -13888
Term 2:  -166.744
Term 3:  36342.7
obj_val:  22287.9
norm_log_prob -40979.27459759252
log_density_val -43208.1
gamma_value [-0.59080136 -2.74391297 -0.23456452  1.56612809 -1.45449092 -2.90130273
  1.00045065 -1.49990151 -1.97265516 -0.59450001 -1.0402721  -0.47671579
 -1.33422536  0.53294927  1.01114906  0.32467001  0.01194545 -1.50321202
  0.28702056  1.61132918 -0.18771085  0.53818759  0.68358976  0.94849772
  0.93073371]
Term 1:  -13888
Term 2:  301.332
Term 3:  36342.7
obj_val:  22756
norm_log_prob -40937.618969

NameError: name 'log_density_function' is not defined

### Plot Results

In [ ]:
# Plot Error Results
plt.plot(results['error_tracker'])
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.show()

NameError: name 'results' is not defined

In [ ]:
# Plot Gamma Estimate Update
for j in range(results['gamma_size']):
    plt.plot(results['gamma_vals_tracker'][:, j], label=r"$\gamma_{%d}$"%(j+1))
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.show()

In [ ]:
# Plot Histograms
for itr in results['collected_ensembles'].keys():
    for j in range(results['gamma_size']):
        plt.hist(results['collected_ensembles'][itr][:, j], bins=100)
        plt.title(f"Iteration {itr+1}; Site {j+1}")
        plt.show()

In [ ]:
# Plot Histogram of the final sample
for j in range(results['gamma_size']):
    plt.hist(results['final_sample'][:, j], bins=100)
    plt.title(f"Final Sample; Site {j+1}")
    plt.show()